## We aim in this notebook to identify duplicates in a CSV containing information about students


In [ ]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
!{sys.executable} -m pip install pandas-profiling


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import nltk
import time
import numpy as np
from scipy import stats


In [ ]:
# from pandas_profiling import ProfileReport

In [102]:
df_students = pd.read_csv("./etudiant.csv")

In [103]:
df_students.head(5)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,id record
0,matthwew,apted,18.0,atherton srteet,currumb in hill,port macquarie,3183,vic,19590825.0,9425976,B_0
1,william,badger,NaN,dwyer s treet,glenlee,west lakes shore,3291,nsw,19001210.0,4656608,B_1
2,connor,bailke,186.0,lambie place,NaN,port lincoln,7303,qld,19670529.0,4702335,B_2
3,kaitlin,goldsworthy,54.0,NaN,NaN,thirroul,2035,nsw,19640517.0,9127277,B_3
4,jasmyn,lowe,48.0,toohey place,grand ecntral,bicton,3085,nsw,19320918.0,1430128,B_4


In [104]:
df_students.columns

Index(['given_name', 'surname', 'street_number', 'address_1', 'address_2',
       'suburb', 'postcode', 'state', 'date_of_birth', 'soc_sec_id',
       'id record'],
      dtype='object')

In [105]:
df_students = df_students.rename(columns={"id record": "record_ID"})


In [106]:
# prof = ProfileReport(df_students)
# prof.to_file(output_file='rapport.html')

In [107]:
df_students[(df_students.surname == 'lowe') & (df_students.given_name=='jasmyn')]

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,record_ID
4,jasmyn,lowe,48.0,toohey place,grand ecntral,bicton,3085,nsw,19320918.0,1430128,B_4
1490,jasmyn,lowe,48.0,toohey place,grand ecntral,bicton,3085,nsw,19320918.0,1430128,B_1490
2341,jasmyn,lowe,48.0,toohey place,grand ecntral,bicton,3085,sw,19320918.0,1430128,B_2341


Here I took a random person's to see if they have a duplicate record. We can see in this case we have two duplicates, one of which has a typo in the state.  

In [108]:
df_students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24940 entries, 0 to 24939
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   given_name     23752 non-null  object 
 1   surname        24412 non-null  object 
 2   street_number  23482 non-null  float64
 3   address_1      23878 non-null  object 
 4   address_2      20698 non-null  object 
 5   suburb         24408 non-null  object 
 6   postcode       24940 non-null  int64  
 7   state          24414 non-null  object 
 8   date_of_birth  23932 non-null  float64
 9   soc_sec_id     24940 non-null  int64  
 10  record_ID      24940 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 2.1+ MB


In [109]:
df_students['date_of_birth'] = pd.to_datetime(df_students['date_of_birth'], format='%Y%m%d', errors='coerce')

In [110]:
df_students['birthyear'] = df_students['date_of_birth'].dt.strftime('%Y').astype(float) ##This will be used later in the blocking part

In [111]:
df_students['birthyear'].median()

1949.0

In [112]:
df_students[df_students['birthyear'].isna()]

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,record_ID,birthyear
23,tylee,berryman,223.0,broad place,villai3,clayton,6355,vic,NaT,2531288,B_23,NaN
30,adam,clarkde,3.0,skinnerstreet,kona,woodcroft,4878,wa,NaT,7112414,B_30,NaN
82,michadl,richmond,7.0,tanumbiriniu street,professional centre,bundaberg,3020,sa,NaT,6836733,B_82,NaN
83,tara,green,25.0,lashbrooke,chisholmstreet,boyne island,3073,nsw,NaT,1388911,B_83,NaN
92,john,cosenza,54.0,wentworth avenue,stoneyq creek,st kilda,4035,NaN,NaT,5193596,B_92,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
24808,kira,reid,6.0,section,saltash place,noth yde,2098,wa,NaT,9065074,B_24808,NaN
24822,jsk,stanbur,25.0,de vis place,kersley mews,koorawatha,4074,qld,NaT,7301101,B_24822,NaN
24848,amber,reid,3.0,hedleystreet,westehram,bibra lake,6108,vin,NaT,9481816,B_24848,NaN
24867,jsk,stanbur,25.0,de vis place,kersley mews,koorawatha,4074,qld,NaT,7301101,B_24867,NaN


In [113]:
df_students['birthyear']= df_students['birthyear'].fillna(df_students['birthyear'].median())


In [114]:
len(df_students[df_students['birthyear'].isna()])

0

In [115]:
df_students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24940 entries, 0 to 24939
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   given_name     23752 non-null  object        
 1   surname        24412 non-null  object        
 2   street_number  23482 non-null  float64       
 3   address_1      23878 non-null  object        
 4   address_2      20698 non-null  object        
 5   suburb         24408 non-null  object        
 6   postcode       24940 non-null  int64         
 7   state          24414 non-null  object        
 8   date_of_birth  23579 non-null  datetime64[ns]
 9   soc_sec_id     24940 non-null  int64         
 10  record_ID      24940 non-null  object        
 11  birthyear      24940 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(7)
memory usage: 2.3+ MB


In [116]:
len(df_students.state.unique())

270

The data is for australian students. However, the number of states is much higher than the actual number of states in Australia.
We have a few issues with the state column:

*   typos
*   postcodes that are not coherant with the states 
*   null values

To resolve this issue, we used the Australian [postcode-state database](https://www.matthewproctor.com/australian_postcodes#downloadlinks) to match the existing postcodes with the corresponding states. 

In [117]:
df_australia = pd.read_csv("./australian_postcodes.csv")

In [118]:
df_australia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18513 entries, 0 to 18512
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 18513 non-null  int64  
 1   postcode           18513 non-null  int64  
 2   locality           18513 non-null  object 
 3   state              18513 non-null  object 
 4   long               18513 non-null  float64
 5   lat                18513 non-null  float64
 6   dc                 17924 non-null  object 
 7   type               16879 non-null  object 
 8   status             18485 non-null  object 
 9   sa3                18380 non-null  float64
 10  sa3name            18380 non-null  object 
 11  sa4                18380 non-null  float64
 12  sa4name            18380 non-null  object 
 13  region             18513 non-null  object 
 14  Lat_precise        18513 non-null  float64
 15  Long_precise       18513 non-null  float64
 16  SA1_MAINCODE_2011  185

In [119]:
df = df_students.rename(columns={"state": "student_state"})


In [120]:
df = df.merge(df_australia[['postcode','state']],on='postcode', how='left')

In [121]:
df

,given_name,surname,street_number,address_1,address_2,suburb,postcode,student_state,date_of_birth,soc_sec_id,record_ID,birthyear,state
0,matthwew,apted,18.0,atherton srteet,currumb in hill,port macquarie,3183,vic,1959-08-25,9425976,B_0,1959.0,VIC
1,matthwew,apted,18.0,atherton srteet,currumb in hill,port macquarie,3183,vic,1959-08-25,9425976,B_0,1959.0,VIC
2,william,badger,NaN,dwyer s treet,glenlee,west lakes shore,3291,nsw,1900-12-10,4656608,B_1,1900.0,NaN
3,connor,bailke,186.0,lambie place,NaN,port lincoln,7303,qld,1967-05-29,4702335,B_2,1967.0,TAS
4,connor,bailke,186.0,lambie place,NaN,port lincoln,7303,qld,1967-05-29,4702335,B_2,1967.0,TAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251119,kobe,tillaer,106.0,stuartstreet,yarrandalne moonee,douglas park,4573,wa,1930-08-31,2715828,B_24939,1930.0,QLD
251120,kobe,tillaer,106.0,stuartstreet,yarrandalne moonee,douglas park,4573,wa,1930-08-31,2715828,B_24939,1930.0,QLD
251121,kobe,tillaer,106.0,stuartstreet,yarrandalne moonee,douglas park,4573,wa,1930-08-31,2715828,B_24939,1930.0,QLD
251122,kobe,tillaer,106.0,stuartstreet,yarrandalne moonee,douglas park,4573,wa,1930-08-31,2715828,B_24939,1930.0,QLD


In [122]:
df = df.drop_duplicates(subset=['record_ID'])

In [123]:
len(df[df['state'].isna()]['state'])

1071

In [124]:
df['state'] = df['state'].fillna(df['student_state'])


<ipython-input-124-8cae9f288354>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['state'] = df['state'].fillna(df['student_state'])


In [125]:
len(df[df['state'].isna()]['state'])

14

In [126]:
df = df.dropna(subset=['state'])

In [127]:
len(df.state.unique())

40

In [128]:
df.groupby("state")["state"].count()

state
ACT      361
NSW     7994
NT       162
QLD     4315
SA      1910
TAS      626
VIC     6213
WA      2288
ac         1
act       14
cic        1
nsj        1
nsl        1
nsm        1
nss        1
nst        1
nsw      389
nswi       1
nt        14
nw         2
nws        8
ql         1
qld      150
qlu        1
qnd        1
sa        68
sk         2
sw         2
tas       51
tfs        1
tsw        1
vic      260
vicb       1
vici       1
vicw       1
vif        1
vix        1
wa        77
wax        1
wp         1
Name: state, dtype: int64

In [129]:
len(df_students.date_of_birth.unique())

4653

In [130]:
len(df_students.state.unique())

270

In [131]:
df_students.groupby("postcode")["postcode"].count()

postcode
216     6
237     2
276     4
278     1
280     8
       ..
8235    2
9205    3
9207    1
9316    5
9319    2
Name: postcode, Length: 1796, dtype: int64

# Blocking method
An important decision for the blocking method is the choice of a good partitioning predicate, which determines the number and the size of the partitions. It should be chosen in a manner that potential duplicates are grouped in the same partition


---


The code below does this for us. In doing so, it uses the following rule:

*   we devided the dataset by states and the by year of birth
*   The edit distance between the attribute surname values of the two records needs to be smaller or equal to 2 (or given_name in case surname is null), and
they need to have the same value for the social security number.


For two records to match, i.e. refer to the same student in the real world:

The edit distance between the attribute name values of the two records needs to be smaller or equal to 2, and
they need to have the same value for the social security number.
Same name, zip code and social security number

In [132]:
df['state'] = df['state'].astype(str)

In [133]:
ACT = ['ACT', 'ac', 'act']
NSW = ['NSW', 'nsw','nswi','nsj','nws','tsw' ]
NT = ['NT', 'nt', 'nw']
QLD =['QLD', 'qld', 'qnd', 'ql', 'qlu']
SA = ['sa','sk','SA']
TAS = ['TAS','tas','tfs','tsw']
VIC = ['VIC','vic','vicb', 'vici', 'vif', 'vix','cic']
WA = ['WA', 'wa', 'wax', 'wp']


In [134]:
df.birthyear.describe()

count    24926.000000
mean      1948.673714
std         28.198943
min       1900.000000
25%       1925.000000
50%       1949.000000
75%       1972.000000
max       1999.000000
Name: birthyear, dtype: float64

In [135]:
blocks = []
block_1 = df[df['state'].str.contains('|'.join(ACT))].sort_values(by="birthyear")
block_1_1 = block_1[block_1['birthyear']<=1925]
block_1_2 = block_1[(block_1['birthyear']<=1950) & (block_1['birthyear']>1925)]
block_1_3 = block_1[(block_1['birthyear']<=1975) & (block_1['birthyear']>1950)]
block_1_4 = block_1[(block_1['birthyear']<=2000) & (block_1['birthyear']>1975)]

block_2 = df[df['state'].str.contains('|'.join(NSW))].sort_values(by="birthyear")
block_2_1 = block_2[block_2['birthyear']<=1925]
block_2_2 = block_2[(block_2['birthyear']<=1950) & (block_2['birthyear']>1925)]
block_2_3 = block_2[(block_2['birthyear']<=1975) & (block_2['birthyear']>1950)]
block_2_4 = block_2[(block_2['birthyear']<=2000) & (block_2['birthyear']>1975)]

block_3 = df[df['state'].str.contains('|'.join(NT))].sort_values(by="birthyear")
block_3_1 = block_3[block_3['birthyear']<=1925]
block_3_2 = block_3[(block_3['birthyear']<=1950) & (block_3['birthyear']>1925)]
block_3_3 = block_3[(block_3['birthyear']<=1975) & (block_3['birthyear']>1950)]
block_3_4 = block_3[(block_3['birthyear']<=2000) & (block_3['birthyear']>1975)]

block_4 = df[df['state'].str.contains('|'.join(QLD))].sort_values(by="birthyear")
block_4_1 = block_4[block_4['birthyear']<=1925]
block_4_2 = block_4[(block_4['birthyear']<=1950) & (block_4['birthyear']>1925)]
block_4_3 = block_4[(block_4['birthyear']<=1975) & (block_4['birthyear']>1950)]
block_4_4 = block_4[(block_4['birthyear']<=2000) & (block_4['birthyear']>1975)]

block_5 = df[df['state'].str.contains('|'.join(SA))].sort_values(by="birthyear")
block_5_1 = block_5[block_5['birthyear']<=1925]
block_5_2 = block_5[(block_5['birthyear']<=1950) & (block_5['birthyear']>1925)]
block_5_3 = block_5[(block_5['birthyear']<=1975) & (block_5['birthyear']>1950)]
block_5_4 = block_5[(block_5['birthyear']<=2000) & (block_5['birthyear']>1975)]

block_6 = df[df['state'].str.contains('|'.join(TAS))].sort_values(by="birthyear")
block_6_1 = block_6[block_6['birthyear']<=1925]
block_6_2 = block_6[(block_6['birthyear']<=1950) & (block_6['birthyear']>1925)]
block_6_3 = block_6[(block_6['birthyear']<=1975) & (block_6['birthyear']>1950)]
block_6_4 = block_6[(block_6['birthyear']<=2000) & (block_6['birthyear']>1975)]

block_7 = df[df['state'].str.contains('|'.join(VIC))].sort_values(by="birthyear")
block_7_1 = block_7[block_7['birthyear']<=1925]
block_7_2 = block_7[(block_7['birthyear']<=1950) & (block_7['birthyear']>1925)]
block_7_3 = block_7[(block_7['birthyear']<=1975) & (block_7['birthyear']>1950)]
block_7_4 = block_7[(block_7['birthyear']<=2000) & (block_7['birthyear']>1975)]

block_8 = df[df['state'].str.contains('|'.join(WA))].sort_values(by="birthyear")
block_8_1 = block_8[block_8['birthyear']<=1925]
block_8_2 = block_8[(block_8['birthyear']<=1950) & (block_8['birthyear']>1925)]
block_8_3 = block_8[(block_8['birthyear']<=1975) & (block_8['birthyear']>1950)]
block_8_4 = block_8[(block_8['birthyear']<=2000) & (block_8['birthyear']>1975)]

blocks.append(block_1_1)
blocks.append(block_1_2) 
blocks.append(block_1_3) 
blocks.append(block_1_4) 

blocks.append(block_2_1)
blocks.append(block_2_2)
blocks.append(block_2_3)
blocks.append(block_2_4)

blocks.append(block_3_1)
blocks.append(block_3_2)
blocks.append(block_3_3)
blocks.append(block_3_4)


blocks.append(block_4_1)
blocks.append(block_4_2)
blocks.append(block_4_3)
blocks.append(block_4_4)


blocks.append(block_5_1)
blocks.append(block_5_2)
blocks.append(block_5_3)
blocks.append(block_5_4)

blocks.append(block_6_1)
blocks.append(block_6_2)
blocks.append(block_6_3)
blocks.append(block_6_4)

blocks.append(block_7_1)
blocks.append(block_7_2)
blocks.append(block_7_3)
blocks.append(block_7_4)

blocks.append(block_8_1)
blocks.append(block_8_2)
blocks.append(block_8_3)
blocks.append(block_8_4)

In [136]:
len(df[df['surname'].isna()]['surname'])

528

In [137]:
len(block_1)+len(block_2)+len(block_3)+len(block_4)+len(block_5)+len(block_6)+len(block_7)+len(block_8)

24929

In [138]:
df.head(10)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,student_state,date_of_birth,soc_sec_id,record_ID,birthyear,state
0,matthwew,apted,18.0,atherton srteet,currumb in hill,port macquarie,3183,vic,1959-08-25,9425976,B_0,1959.0,VIC
2,william,badger,NaN,dwyer s treet,glenlee,west lakes shore,3291,nsw,1900-12-10,4656608,B_1,1900.0,nsw
3,connor,bailke,186.0,lambie place,NaN,port lincoln,7303,qld,1967-05-29,4702335,B_2,1967.0,TAS
10,kaitlin,goldsworthy,54.0,NaN,NaN,thirroul,2035,nsw,1964-05-17,9127277,B_3,1964.0,NSW
13,jasmyn,lowe,48.0,toohey place,grand ecntral,bicton,3085,nsw,1932-09-18,1430128,B_4,1932.0,VIC
16,joshua,coleman,16.0,mosman place,aarlee,koouong,3432,nsw,1950-10-24,2640319,B_5,1950.0,VIC
17,calbe,newberry,14.0,mackenziestreet,mountainview retirement vlge,emerald,2560,sa,1921-12-03,1454125,B_6,1921.0,NSW
36,beloma,griersn,27.0,priddle street,green lagoon,christiesbeach,3152,vic,1931-06-23,5872915,B_7,1931.0,VIC
40,robbie,carmody,34.0,bernacchi street,melita,east fremantle,2170,qld,1939-12-11,8441818,B_8,1939.0,NSW
52,NaN,tholrn,66.0,keithstreet,ocean sta r villas,beenleigh,6107,vic,1955-07-03,6047972,B_9,1955.0,WA


Now that we have the blocks, we have to set rules for finding the matches:
-same social security number, date of birth, (surname,name)

In [139]:
num_records = len(df)
matches = []
number_of_matches = 0
start = time.process_time()
for block in blocks:
    num_records = len(block)
    print("num_records of the block is {}".format(num_records))
    for i in range(0,num_records):
        for j in range(i+1,num_records):
          if (not pd.isnull(block.iloc[i,0])) and (not pd.isnull(block.iloc[j,0])):
            score = nltk.edit_distance(block.iloc[i,0], block.iloc[j,0])
          elif not (pd.isnull(block.iloc[i,1]) or pd.isnull(block.iloc[j,1])):
            score = nltk.edit_distance(block.iloc[i,1], block.iloc[j,1])
          else:
            score= nltk.edit_distance(str(block.iloc[i,6]), str(block.iloc[j,6]))

          age_score = nltk.edit_distance(str(block.iloc[i,11]), str(block.iloc[j,11]))
          soc_sec_score = nltk.edit_distance(str(block.iloc[i,9]), str(block.iloc[j,9]))
          if (soc_sec_score <= 1) and (age_score<= 1) and (score<=1):
                number_of_matches = number_of_matches + 1 
                matches.append((block.iloc[i,10],block.iloc[j,10]))

print("Number of matches: {}".format(number_of_matches))

num_records of the block is 73
num_records of the block is 140
num_records of the block is 93
num_records of the block is 70
num_records of the block is 2244
num_records of the block is 2358
num_records of the block is 2006
num_records of the block is 1786
num_records of the block is 38
num_records of the block is 51
num_records of the block is 29
num_records of the block is 68
num_records of the block is 1152
num_records of the block is 1213
num_records of the block is 1103
num_records of the block is 1000
num_records of the block is 482
num_records of the block is 625
num_records of the block is 425
num_records of the block is 448
num_records of the block is 167
num_records of the block is 190
num_records of the block is 147
num_records of the block is 175
num_records of the block is 1714
num_records of the block is 1866
num_records of the block is 1467
num_records of the block is 1432
num_records of the block is 595
num_records of the block is 712
num_records of the block is 525
num